# Megaron

In [317]:
#izberi dan, od kdaj do kdaj, dobi datume,dobi datume

In [318]:
import time
import datetime
import pandas as pd
import numpy as np
from datetime import timezone
import csv
from hungarian import Hungarian

In [319]:
def min_max_timestamp(leto):
    min_timestamp = datetime.datetime(leto,1,1).replace(tzinfo=timezone.utc).timestamp()
    max_timestamp = datetime.datetime(leto,12,31).replace(tzinfo=timezone.utc).timestamp()
    return (min_timestamp,max_timestamp)

def min_max_timestamp_schoolyear(leto_zacetek):
    '''nekako studentsko solsko leto'''
    min_timestamp = (datetime.datetime(leto_zacetek,10,1).replace(tzinfo=timezone.utc).timestamp())
    max_timestamp = (datetime.datetime(leto_zacetek+1,6,25).replace(tzinfo=timezone.utc).timestamp())
    return (min_timestamp,max_timestamp)

def gen_days(min_timestamp, max_timestamp, iso_weekday):
    '''creates a list of every day between timestamps, that is the weekday (1 monday, 7 sunday)'''
    
    day = 60*60*24
    week = day*7
    
    min_weekday = datetime.datetime.fromtimestamp(min_timestamp).replace(tzinfo=timezone.utc).isoweekday()
    diff = iso_weekday-min_weekday
    if (diff < 0):
        diff += 7
    
    timestamp_list = []
    timestamp  = min_timestamp + diff*day
    while(timestamp <= max_timestamp):
        timestamp_list.append(timestamp)
        timestamp += week
    return timestamp_list

In [320]:
a = min_max_timestamp_schoolyear(2018)
vaje = [int(x) for x in gen_days(a[0],a[1],2)]
#vaje_pandas = pd.to_datetime(vaje, unit='s')

In [321]:
#preberemo rojstne dneve
df = pd.read_csv('lanski.csv', delimiter=';', names = ['name','date'])
leto = datetime.datetime.now().year
df['day'] = df['date'].astype(str).str[:2]
df['month'] = df['date'].astype(str).str[3:5]
df['year'] = np.where((df['month'].astype(int) < 8),leto+1,leto)
df['timestamp'] = pd.to_datetime(df[['year', 'month', 'day']])
ljudje = df[['name', 'timestamp']].copy()

In [322]:
imena = ljudje['name'].values.tolist()
rojstni_dnevi = [ int(x / 1000000000) for x in ljudje['timestamp'].values.tolist()]

In [323]:
#ker castita dva na vajo podvojimo vsak element
vaje_vozlisca = [x for x in vaje for _ in (0, 1)]

In [324]:
#vsak stolpec ena vaja, vrstica en clovek
matrika = [[int(abs(vaja-clovek)/86400) for vaja in vaje_vozlisca] for clovek in rojstni_dnevi]

In [325]:
matching = Hungarian(matrika)
matching.calculate()

In [326]:
result = matching.get_results()
potential = matching.get_total_potential()

In [327]:
result = sorted(result,key=lambda tup: tup[1])

In [328]:
with open('razpored.csv', 'w') as csvfile:
    fieldnames = ['vaja', 'ime','rojstni_dan','dni']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,delimiter = ';')

    writer.writeheader()
    for par in result:
        ime = imena[par[0]]
        vaja = datetime.datetime.fromtimestamp(vaje_vozlisca[par[1]]).replace(tzinfo=timezone.utc).date()
        rojstni_dan = datetime.datetime.fromtimestamp(rojstni_dnevi[par[0]]).replace(tzinfo=timezone.utc).date()
        razdalja = abs(rojstni_dan-vaja).days
        #print(ime,str(vaja),str(rojstni_dan),razdalja)
        writer.writerow({'vaja': vaja, 'ime': ime, 'rojstni_dan' : rojstni_dan, 'dni': razdalja})

In [329]:
potential

1224